## Web Scraping para extraer información de todos los personajes del Manga <br> *JoJo's Bizarre Adventure (Parte 1-8)*

In [1]:
#Importando Librerías
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import warnings
warnings.filterwarnings("ignore")
# pd.set_option('display.max_rows', None)

In [2]:
#Creando WebDriver para la busqueda con Chrome
driver = webdriver.Chrome(ChromeDriverManager().install())

#Ingresando a página para obtener información de las partes de Jojo's
page_url = "https://jojo.fandom.com/es/wiki/Categor%C3%ADa:Partes_de_JoJo%27s_Bizarre_Adventure"
driver.get(page_url)

#Encontrando partes
jojo_parts = driver.find_elements(By.XPATH,'//*[@id="mw-content-text"]/div[3]/div/ul/li/a')

#Extrayendo información Jojo's(Parte - URL)
jojo_titles = []
for parts in jojo_parts:
    
    #Extrayendo nombre de la parte
    title = parts.text
    #Extrayendo link donde se encuentra la tabla con los personajes
    url = parts.get_attribute('href')
    jojo_titles.append({'title':title,'url':url})
    
driver.quit()

[WDM] - Downloading: 100%|██████████| 6.21M/6.21M [00:07<00:00, 817kB/s] 


In [3]:
#Derificando data extraida (Parte - URL)
jojo_titles

[{'title': 'Battle Tendency',
  'url': 'https://jojo.fandom.com/es/wiki/Battle_Tendency'},
 {'title': 'Diamond is Unbreakable',
  'url': 'https://jojo.fandom.com/es/wiki/Diamond_is_Unbreakable'},
 {'title': 'JoJo Lands', 'url': 'https://jojo.fandom.com/es/wiki/JoJo_Lands'},
 {'title': 'JoJolion', 'url': 'https://jojo.fandom.com/es/wiki/JoJolion'},
 {'title': 'Phantom Blood',
  'url': 'https://jojo.fandom.com/es/wiki/Phantom_Blood'},
 {'title': 'Stardust Crusaders',
  'url': 'https://jojo.fandom.com/es/wiki/Stardust_Crusaders'},
 {'title': 'Steel Ball Run',
  'url': 'https://jojo.fandom.com/es/wiki/Steel_Ball_Run'},
 {'title': 'Stone Ocean',
  'url': 'https://jojo.fandom.com/es/wiki/Stone_Ocean'},
 {'title': 'Vento Aureo',
  'url': 'https://jojo.fandom.com/es/wiki/Vento_Aureo'}]

In [4]:
#Lista de Tablas
characters_list = []

for value in jojo_titles:
    #Lectura de tablas con datos de personajes
    df = pd.read_html(value['url'])
    # print(df)
    for i in range(len(df)):
        if df[i].columns[0] in ['Personaje', 'Personajes','Nombre']:
            #Agregando columna para identificar parte
            temp = df[i].copy()
            temp.insert(0,'Parte',value['title'])
            
            #Agregando tabla a la lista
            characters_list.append(temp)
            
#Uniendo tablas de diferentes partes para próxima limpieza
characters = pd.concat(characters_list).reset_index(drop=True)

In [5]:
characters

,Parte,Personaje,Relación,Nombre,Stand,Rol,Personajes
0,Battle Tendency,Joseph Joestar,protagonista,NaN,NaN,NaN,NaN
1,Battle Tendency,Smokey Brown,aliado,NaN,NaN,NaN,NaN
2,Battle Tendency,Erina Joestar,aliado,NaN,NaN,NaN,NaN
3,Battle Tendency,Robert Edward O. Speedwagon,aliado,NaN,NaN,NaN,NaN
4,Battle Tendency,Rudol von Stroheim,aliado (antes antagonista),NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
225,Vento Aureo,Cioccolata,NaN,NaN,Green Day,Antagonista,NaN
226,Vento Aureo,Secco,NaN,NaN,Oasis,Antagonista,NaN
227,Vento Aureo,Scolippi,NaN,NaN,Rolling Stones,Neutral,NaN
228,Vento Aureo,Jean Pierre Polnareff,NaN,NaN,Silver Chariot - Chariot Requiem,Aliado,NaN


In [6]:
#Limpieza de datos por que las columnas tienen nombres diferentes
characters['Relación'] = characters['Relación'].fillna('') + ('' + characters['Rol']).fillna('')
characters['Personaje'] = characters['Personaje'].fillna('') + ('' + characters['Personajes']).fillna('')
characters['Personaje'] = characters['Personaje'].fillna('') + ('' + characters['Nombre']).fillna('')

#Eliminando columnas ya innecesarias
characters.drop(columns=['Rol'], inplace=True)
characters.drop(columns=['Personajes'], inplace=True)
characters.drop(columns=['Nombre'], inplace=True)
# characters.replace({np.nan:''}, inplace=True)

In [7]:
characters

,Parte,Personaje,Relación,Stand
0,Battle Tendency,Joseph Joestar,protagonista,NaN
1,Battle Tendency,Smokey Brown,aliado,NaN
2,Battle Tendency,Erina Joestar,aliado,NaN
3,Battle Tendency,Robert Edward O. Speedwagon,aliado,NaN
4,Battle Tendency,Rudol von Stroheim,aliado (antes antagonista),NaN
...,...,...,...,...
225,Vento Aureo,Cioccolata,Antagonista,Green Day
226,Vento Aureo,Secco,Antagonista,Oasis
227,Vento Aureo,Scolippi,Neutral,Rolling Stones
228,Vento Aureo,Jean Pierre Polnareff,Aliado,Silver Chariot - Chariot Requiem


In [8]:
#Dando formato Title a todo el Dataframe
characters[characters.columns] = characters[characters.columns].apply(
    lambda x: x.str.title() if x.dtype == "object" else x)

#Corrigiendo registro
characters['Personaje'][characters['Stand']=='Anubis'] = 'Caravan Serai, Chaka, Khan, Jean Pierre Polnareff'

In [9]:
characters

,Parte,Personaje,Relación,Stand
0,Battle Tendency,Joseph Joestar,Protagonista,NaN
1,Battle Tendency,Smokey Brown,Aliado,NaN
2,Battle Tendency,Erina Joestar,Aliado,NaN
3,Battle Tendency,Robert Edward O. Speedwagon,Aliado,NaN
4,Battle Tendency,Rudol Von Stroheim,Aliado (Antes Antagonista),NaN
...,...,...,...,...
225,Vento Aureo,Cioccolata,Antagonista,Green Day
226,Vento Aureo,Secco,Antagonista,Oasis
227,Vento Aureo,Scolippi,Neutral,Rolling Stones
228,Vento Aureo,Jean Pierre Polnareff,Aliado,Silver Chariot - Chariot Requiem


In [10]:
characters.Parte.unique()

array(['Battle Tendency', 'Diamond Is Unbreakable', 'Jojolion',
       'Phantom Blood', 'Stardust Crusaders', 'Steel Ball Run',
       'Stone Ocean', 'Vento Aureo'], dtype=object)

In [11]:
parts = ['Phantom Blood', 'Battle Tendency', 'Stardust Crusaders',
             'Diamond Is Unbreakable', 'Vento Aureo', 'Stone Ocean', 'Steel Ball Run', 'Jojolion']

In [14]:
characters = characters.set_index('Parte')
characters = characters.loc[parts].reset_index(drop=True)
characters

In [15]:
characters.sort_values('Personaje')

,Parte,Personaje,Relación,Stand
67,Jojolion,Aisho Dainenjiyama,Antagonista,Doobie Wah!
44,Diamond Is Unbreakable,Akira Otoishi,Antagonista,Red Hot Chili Pepper
130,Stardust Crusaders,Alessi,Antagonista,Sethan
149,Steel Ball Run,Andre Boom Boom,Antagonista,Tomb Of The Boom
43,Diamond Is Unbreakable,Anjuro Katagiri,Antagonista,Aqua Necklace
...,...,...,...,...
66,Jojolion,Yotsuyu Yagiyama,Antagonista,I Am A Rock
26,Diamond Is Unbreakable,Yukako Yamagishi,Aliado (Antes Antagonista),Love Deluxe
32,Diamond Is Unbreakable,Yuya Fungami,Neutral (Antes Antagonista),Highway Star
118,Stardust Crusaders,Zz,Antagonista,Wheel Of Fortune


In [16]:
characters[characters['Personaje'].str.contains('Jotaro')]

,Parte,Personaje,Relación,Stand
20,Diamond Is Unbreakable,Jotaro Kujo,Aliado,Star Platinum
100,Stardust Crusaders,Jotaro Kujo,Protagonista Principal,Star Platinum
177,Stone Ocean,Jotaro Kujo,Aliado,Star Platinum
207,Vento Aureo,Jotaro Kujo,Aliado (De Koichi),Star Platinum (No Visto)


In [17]:
#Creando CSV con la data obtenida
characters.to_csv('jojo_characters.csv', index=False, encoding='utf-8-sig')

In [18]:
#Creando Excel con la data obtenida
characters.to_excel('jojo_characters.xlsx', index=False, encoding='utf-8-sig')